# Let us begin with a classical approach to the Shor's Algorithm

In [117]:
import random
import math

In [118]:
# Primality test (using the simple 6k +/- 1 optimization)
def is_prime(n):
    if n <= 3:
       return n > 1
    if n % 2 == 0 or n % 3 == 0:
       return False
    i = 5
    while i * i <= n:
       if n % i == 0 or n % (i + 2) == 0:
            return False
       i += 6
    return True

In [119]:
# GCD (Euclidean algorithm)
def gcd(a, b):
    while b:
       a, b = b, a % b
    return a

In [120]:
# Modular exponentiation to prevent overflow
def pow_mod(x, y, z):
    result = 1
    x = x % z
    while y > 0:
       if y & 1:
            result = (result * x) % z
       y >>= 1
       x = (x * x) % z
    return result

In [121]:
# Classical period finding (inefficient compared to quantum approach)
def find_period(a, N):
    r = 1
    while pow_mod(a, r, N) != 1:
       r += 1
    return r

In [122]:
# Check if the number is a power of another number
def check_power(N):
    for a in range(2, int(math.log2(N)) + 1):
       b = round(N ** (1 / a))
       if b ** a == N:
          return True, a, b
    return False, None, None

In [123]:
# Shor's algorithm (classical simulation)
def shors_algorithm(N):
    assert N >= 2
    # Step 1: Check if N is prime or a power of a number
    if is_prime(N):
       return N
    is_power, a, b = check_power(N)
    if is_power:
       return b

    while True:
       # Step 2: Choose a random x in [2, N - 1]
       x = random.randint(2, N - 1)
       # Step 3: Compute gcd(x, N)
       x_gcd = gcd(x, N)
       if x_gcd != 1:
            # Found a factor
            return x_gcd

       # Step 4: Find period r using classical period finding
       r = find_period(x, N)
       if r % 2 == 1:
          # If r is odd, try again
          continue

       # Step 5: Compute factors
       factor1 = gcd(pow_mod(x, r // 2, N) - 1, N)
       factor2 = gcd(pow_mod(x, r // 2, N) + 1, N) 
       if factor1 == N or factor2 == N:
          # Unlucky choice of x, try again
          continue

       # Return the non-trivial factors
       return factor1, factor2

In [124]:
def find_factors(N):
    factors = []
    factor = shors_algorithm(N)
    if str(type(factor)) == "<class 'int'>":
        factors.append(factor)
        factors.append(N/factor)
        if is_prime(factor) == False: 
                factors.append(find_factors(factor))
        if is_prime(int(N/factor)) == False: 
            factors.append(find_factors(int(N/factor)))
    else: 
        for i in range(len(factor)):
            factors.append(factor[i])
            factors.append(N/factor[i])
            if is_prime(factor[i]) == False: 
                factors.append(find_factors(factor[i]))
            if is_prime(int(N/factor[i])) == False: 
                factors.append(find_factors(int(N/factor[i])))
    return factors

In [125]:
def flatten_array(arr):
    flattened = []
   
    for element in arr:
       if isinstance(element, list): # Check if it's a list
            # If it is a list, extend the flattened list with the result
            # of a recursive call to flatten the sublist
          flattened.extend(flatten_array(element))
       else:
            # If it's not a list, just append the element
            flattened.append(element)
   
    return flattened

# Example array
#arr = [6, 5.0, [3, 2.0], 10, 3.0, [2, 5.0]]

# Flattening the array
#flattened_arr = flatten_array(arr)
#print(flattened_arr)

In [126]:
def clean_arr(arr):
    #flatten array
    arr = flatten_array(arr)
    
    # Step 1: Typecast to integers
    arr = [int(number) for number in arr]
   
    # Step 2: Remove duplicates
    arr = list(set(arr))
   
    # Step 3: Filter out non-prime numbers
    prime_arr = [num for num in arr if is_prime(num)]
   
    return prime_arr

# Example array
# arr = [6, 5.0, [3, 2.0], 10, 3.0, [2, 5.0]]

# Cleaning the array
# cleaned_arr = clean_arr(arr)
# print(cleaned_arr)

In [127]:
# Example usage:
N = 33 # Example number to factor
factors = find_factors(N)
print(clean_arr(factors))

[11, 3]
